In [ ]:
!pip install --upgrade azureml-sdk

In [ ]:
import json
import logging

from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import os

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.interpret import ExplanationClient

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

In [ ]:
ws = Workspace(subscription_id='c463a189-ff51-4b8d-aa64-edea8b0527e5',
               resource_group='Diabetes',
               workspace_name='Diabetes1')

# Choose a name for the experiment.
experiment_name = 'Diabetes'

experiment = Experiment(ws, experiment_name)

output = {}
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Run History Name'] = experiment_name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

,
Subscription ID,c463a189-ff51-4b8d-aa64-edea8b0527e5
Workspace,Diabetes1
Resource Group,Diabetes
Location,centralindia
Run History Name,Diabetes


In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "cpu-cluster-4"

try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [ ]:
data = pd.read_csv('/content/data/diabetes.csv')

if not os.path.isdir('data'):
    os.mkdir('data')
    
pd.DataFrame(data).to_csv("data/train_data.csv", index=False)

ds = ws.get_default_datastore()
ds.upload(src_dir='./data', target_path='first_disease', overwrite=True, show_progress=True)

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 2 files
Uploading ./data/train_data.csv
Uploaded ./data/train_data.csv, 1 files out of an estimated total of 2
Uploading ./data/diabetes.csv
Uploaded ./data/diabetes.csv, 2 files out of an estimated total of 2
Uploaded 2 files


$AZUREML_DATAREFERENCE_c466f615b3b44a8ea84bc7068b5c1e5c

In [ ]:
train_data = Dataset.Tabular.from_delimited_files(path=ds.path('first_disease/train_data.csv'))

In [ ]:
automl_settings = {
    "experiment_timeout_hours" : 0.6,
    "enable_early_stopping" : True,
    "iteration_timeout_minutes": 5,
    "max_concurrent_iterations": 4,
    "max_cores_per_iteration": -1,
    "n_cross_validations": 2,
    "primary_metric": 'accuracy',
    "featurization": 'auto',
    "verbosity": logging.INFO,
}

automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors.log',
                             compute_target=compute_target,
                             experiment_exit_score = 0.9984,
                             blocked_models = ['KNN','LinearSVM'],
                             enable_onnx_compatible_models=True,
                             validation_size=0.2,
                             training_data = train_data,
                             label_column_name = 'Outcome',
                             **automl_settings
                            )

In [ ]:
remote_run = experiment.submit(automl_config, show_output=False)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
Diabetes,AutoML_51f04180-9434-4fa3-b671-263cf2c086f2,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
import pickle

pickle_out = open("Diabetes_model.pkl", "wb") 
pickle.dump(fitted_model, pickle_out) 
pickle_out.close()